### DP0701EN-Week3-PeerGraded Project - Part II

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Use the Geocoder package or the csv file to create the following dataframe:

In [1]:
import pandas as pd
from IPython.display import IFrame
import numpy as np
!pip install lxml

In [2]:
#import html
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
IFrame(url, width = 800, height = 350)

In [3]:
#read dataframe from html
df, = pd.read_html(url, match = "Postcode", skiprows = 1)
df.columns = ["PostalCode", "Borough", "Neighborhood"]
#Ignore cells with a borough that is Not assigned
df = df[df["Borough"] != "Not assigned"]
#combined two rows into one row with the neighborhoods separated with a comma
df.index = np.arange(0, len(df))
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(list)
df = df.sample(frac = 1).reset_index()
df["Neighborhood"] = df["Neighborhood"].str.join(", ")
#If a cell has a borough but a Not assigned neighborhood, 
#then the neighborhood will be the same as the borough
for index, row in df.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
df.head()

,PostalCode,Borough,Neighborhood
0,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."
1,M2J,North York,"Fairview, Henry Farm, Oriole"
2,M4L,East Toronto,"The Beaches West, India Bazaar"
3,M6C,York,Humewood-Cedarvale
4,M4P,Central Toronto,Davisville North


In [4]:
# find latitude and longitude
#!pip install geocoder
import geocoder # import geocoder

df["Latitude"] = 0
df["Longitude"] = 0

# loop until you get the coordinates - Geocoder
#for index, row in df.iterrows():
 #   lat_lng_coords = None
 #   postal_code = row["PostalCode"]
 #   while(lat_lng_coords is None):
 #       g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
 #       lat_lng_coords = g.latlng
 #
 #   row["Latitude"] = lat_lng_coords[0]
 #  row["Longitude"] = lat_lng_coords[1]


In [5]:
#Alternative Method
df_coordinates = pd.read_csv("Geospatial_Coordinates.csv")
df_coordinates.columns = ["PostalCode","Latitude","Longitude"]
df_coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
for index, row in df.iterrows():
    df_coordinates_temp = df_coordinates[df_coordinates["PostalCode"] == row["PostalCode"]]
    df_coordinates_temp.reset_index(inplace = True)
    df.loc[index, "Latitude"] = df_coordinates_temp.loc[0, "Latitude"]
    df.loc[index, "Longitude"] = df_coordinates_temp.loc[0, "Longitude"]
  
df.to_csv("Week3project.csv")
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
4,M4P,Central Toronto,Davisville North,43.712751,-79.390197


In [7]:
df.shape

(103, 5)

This conludes part 2.